In [1]:
import graphlab as gl

In [2]:
filename = "./price_EUR_USD_*H1.csv"
sf_eur_usd_h1 = gl.SFrame.read_csv(filename)

[INFO] This non-commercial license of GraphLab Create is assigned to yuecong1104@gmail.comand will expire on August 13, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-4547 - Server binary: /Library/Python/2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1439795994.log
[INFO] GraphLab Server Version: 1.5.2


PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2002-01-01T00%3A00%3A00Z_H1.csv
PROGRESS: Read 48 lines. Lines per second: 463.992
PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2002-07-01T00%3A00%3A00Z_H1.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.104006 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int,int,float,float,float,float,float,float,float,float,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2002-01-01T00%3A00%3A00Z_H1.csv
PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2002-07-01T00%3A00%3A00Z_H1.csv
PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2003-

In [3]:
#sf_eur_usd_h1.show()

In [4]:
sf_eur_usd_h1.add_column( sf_eur_usd_h1['year'] .astype(str)+ '-' + sf_eur_usd_h1['month'].astype(str) + 
                         '-' + sf_eur_usd_h1['day'].astype(str) , 'date' )
sf_eur_usd_h1.add_column((sf_eur_usd_h1['openBid'] + sf_eur_usd_h1['openAsk']) /2 , 'open')
sf_eur_usd_h1.add_column((sf_eur_usd_h1['highBid'] + sf_eur_usd_h1['highAsk']) /2 , 'high')
sf_eur_usd_h1.add_column((sf_eur_usd_h1['lowBid'] + sf_eur_usd_h1['lowAsk']) /2 , 'low')
sf_eur_usd_h1.add_column((sf_eur_usd_h1['closeBid'] + sf_eur_usd_h1['closeAsk']) /2 , 'close')
sf_eur_usd_h1.remove_columns(['openBid','openAsk','highBid','highAsk','lowBid','lowAsk','closeBid','closeAsk','complete'])

year,month,day,hour,minute,second,volume,date,open,high,low,close
2002,5,6,20,0,0,1,2002-5-6,0.91825,0.91825,0.91825,0.91825
2002,5,7,20,0,0,1,2002-5-7,0.91535,0.91535,0.91535,0.91535
2002,5,8,20,0,0,1,2002-5-8,0.90435,0.90435,0.90435,0.90435
2002,5,9,20,0,0,1,2002-5-9,0.90935,0.90935,0.90935,0.90935
2002,5,10,20,0,0,1,2002-5-10,0.914,0.914,0.914,0.914
2002,5,10,22,0,0,1,2002-5-10,0.9141,0.9141,0.9141,0.9141
2002,5,13,20,0,0,1,2002-5-13,0.91155,0.91155,0.91155,0.91155
2002,5,14,20,0,0,1,2002-5-14,0.90275,0.90275,0.90275,0.90275
2002,5,15,20,0,0,1,2002-5-15,0.91255,0.91255,0.91255,0.91255
2002,5,16,20,0,0,1,2002-5-16,0.91195,0.91195,0.91195,0.91195


In [5]:
sf_eur_usd_h1 = sf_eur_usd_h1.sort(['year','month','day','hour','minute','second'])

In [6]:
sf_eur_usd_h1_tmp = sf_eur_usd_h1

In [7]:
sf_eur_usd_h1_tmp.add_column(sf_eur_usd_h1_tmp['open'] *sf_eur_usd_h1_tmp['volume'] ,'open_volume')
sf_eur_usd_h1_tmp.add_column(sf_eur_usd_h1_tmp['high'] *sf_eur_usd_h1_tmp['volume'] ,'high_volume')
sf_eur_usd_h1_tmp.add_column(sf_eur_usd_h1_tmp['low'] *sf_eur_usd_h1_tmp['volume'] ,'low_volume')
sf_eur_usd_h1_tmp.add_column(sf_eur_usd_h1_tmp['close'] *sf_eur_usd_h1_tmp['volume'] ,'close_volume')

year,month,day,hour,minute,second,volume,date,open,high,low,close,open_volume
2002,5,6,20,0,0,1,2002-5-6,0.91825,0.91825,0.91825,0.91825,0.91825
2002,5,7,20,0,0,1,2002-5-7,0.91535,0.91535,0.91535,0.91535,0.91535
2002,5,8,20,0,0,1,2002-5-8,0.90435,0.90435,0.90435,0.90435,0.90435
2002,5,9,20,0,0,1,2002-5-9,0.90935,0.90935,0.90935,0.90935,0.90935
2002,5,10,20,0,0,1,2002-5-10,0.914,0.914,0.914,0.914,0.914
2002,5,10,22,0,0,1,2002-5-10,0.9141,0.9141,0.9141,0.9141,0.9141
2002,5,13,20,0,0,1,2002-5-13,0.91155,0.91155,0.91155,0.91155,0.91155
2002,5,14,20,0,0,1,2002-5-14,0.90275,0.90275,0.90275,0.90275,0.90275
2002,5,15,20,0,0,1,2002-5-15,0.91255,0.91255,0.91255,0.91255,0.91255
2002,5,16,20,0,0,1,2002-5-16,0.91195,0.91195,0.91195,0.91195,0.91195


In [8]:
sf_eur_usd_h1_tmp.show()

Canvas is accessible via web browser at the URL: http://localhost:50244/index.html
Opening Canvas in default web browser.


In [9]:
sf_eur_usd_h1_tmp[sf_eur_usd_h1_tmp['volume'] >100].shape

(66493, 16)

In [10]:
sf_eur_usd_h1_tmp.shape

(69296, 16)

In [11]:
sf_eur_usd_d1_tmp = sf_eur_usd_h1_tmp.groupby(['day','month','year','date'],
                                              [gl.aggregate.SUM('open_volume'),
                                               gl.aggregate.SUM('high_volume'),
                                               gl.aggregate.SUM('low_volume'),
                                               gl.aggregate.SUM('close_volume'),
                                               gl.aggregate.SUM('volume'),
                                              ]
                                             )

In [12]:
sf_eur_usd_d1_tmp = sf_eur_usd_d1_tmp.sort(['year','month','day'])

In [13]:
sf_eur_usd_d1_tmp

date,day,month,year,Sum of open_volume,Sum of high_volume,Sum of low_volume,Sum of close_volume,Sum of volume
2002-5-6,6,5,2002,0.91825,0.91825,0.91825,0.91825,1
2002-5-7,7,5,2002,0.91535,0.91535,0.91535,0.91535,1
2002-5-8,8,5,2002,0.90435,0.90435,0.90435,0.90435,1
2002-5-9,9,5,2002,0.90935,0.90935,0.90935,0.90935,1
2002-5-10,10,5,2002,1.8281,1.8281,1.8281,1.8281,2
2002-5-13,13,5,2002,0.91155,0.91155,0.91155,0.91155,1
2002-5-14,14,5,2002,0.90275,0.90275,0.90275,0.90275,1
2002-5-15,15,5,2002,0.91255,0.91255,0.91255,0.91255,1
2002-5-16,16,5,2002,0.91195,0.91195,0.91195,0.91195,1
2002-5-17,17,5,2002,0.9209,0.9209,0.9209,0.9209,1


In [14]:
sf_eur_usd_d1_tmp.add_column(sf_eur_usd_d1_tmp['Sum of open_volume'] /sf_eur_usd_d1_tmp['Sum of volume'] ,'open')
sf_eur_usd_d1_tmp.add_column(sf_eur_usd_d1_tmp['Sum of high_volume'] /sf_eur_usd_d1_tmp['Sum of volume'] ,'high')
sf_eur_usd_d1_tmp.add_column(sf_eur_usd_d1_tmp['Sum of low_volume'] /sf_eur_usd_d1_tmp['Sum of volume'] ,'low')
sf_eur_usd_d1_tmp.add_column(sf_eur_usd_d1_tmp['Sum of close_volume'] /sf_eur_usd_d1_tmp['Sum of volume'] ,'close')
sf_eur_usd_d1_tmp.rename({'Sum of volume': 'volume'})
sf_eur_usd_d1_tmp.remove_columns(['Sum of open_volume','Sum of high_volume','Sum of low_volume','Sum of close_volume'])

date,day,month,year,volume,open,high,low,close
2002-5-6,6,5,2002,1,0.91825,0.91825,0.91825,0.91825
2002-5-7,7,5,2002,1,0.91535,0.91535,0.91535,0.91535
2002-5-8,8,5,2002,1,0.90435,0.90435,0.90435,0.90435
2002-5-9,9,5,2002,1,0.90935,0.90935,0.90935,0.90935
2002-5-10,10,5,2002,2,0.91405,0.91405,0.91405,0.91405
2002-5-13,13,5,2002,1,0.91155,0.91155,0.91155,0.91155
2002-5-14,14,5,2002,1,0.90275,0.90275,0.90275,0.90275
2002-5-15,15,5,2002,1,0.91255,0.91255,0.91255,0.91255
2002-5-16,16,5,2002,1,0.91195,0.91195,0.91195,0.91195
2002-5-17,17,5,2002,1,0.9209,0.9209,0.9209,0.9209


In [15]:
sf_eur_usd_d1 = sf_eur_usd_d1_tmp

# Use D1 to calculate features, h1 also could be used to calculate features with same logic
sf_eur_usd_features = sf_eur_usd_d1 

In [16]:
pip_unit = 10000
close_list = sf_eur_usd_features ['close']
def close_minus_n_time_unit(time_unit):
    if time_unit ==1:
        close_minus_n_time_unit_list = gl.SArray([close_list[0]]).append(close_list[:-1*time_unit])
    elif time_unit >1:
        close_minus_n_time_unit_list = close_list[0: time_unit].append(close_list[:-1*time_unit])
    else:
        print("close_minus_n_time_unit() parameter error!")
    return close_minus_n_time_unit_list

close_minus_1 = close_minus_n_time_unit(1)
close_minus_2 = close_minus_n_time_unit(2)
close_minus_3 = close_minus_n_time_unit(3)

In [17]:
sf_eur_usd_features.add_column((close_list - close_minus_1) *pip_unit ,'pips')

date,day,month,year,volume,open,high,low,close,pips
2002-5-6,6,5,2002,1,0.91825,0.91825,0.91825,0.91825,0.0
2002-5-7,7,5,2002,1,0.91535,0.91535,0.91535,0.91535,-29.0
2002-5-8,8,5,2002,1,0.90435,0.90435,0.90435,0.90435,-110.0
2002-5-9,9,5,2002,1,0.90935,0.90935,0.90935,0.90935,50.0
2002-5-10,10,5,2002,2,0.91405,0.91405,0.91405,0.91405,47.0
2002-5-13,13,5,2002,1,0.91155,0.91155,0.91155,0.91155,-25.0
2002-5-14,14,5,2002,1,0.90275,0.90275,0.90275,0.90275,-88.0
2002-5-15,15,5,2002,1,0.91255,0.91255,0.91255,0.91255,98.0
2002-5-16,16,5,2002,1,0.91195,0.91195,0.91195,0.91195,-6.0
2002-5-17,17,5,2002,1,0.9209,0.9209,0.9209,0.9209,89.5


In [18]:
predict_pips_list = sf_eur_usd_features['pips'][1:].append(gl.SArray([0.0]))
sf_eur_usd_features.add_column(predict_pips_list,'predict pips')

date,day,month,year,volume,open,high,low,close,pips,predict pips
2002-5-6,6,5,2002,1,0.91825,0.91825,0.91825,0.91825,0.0,-29.0
2002-5-7,7,5,2002,1,0.91535,0.91535,0.91535,0.91535,-29.0,-110.0
2002-5-8,8,5,2002,1,0.90435,0.90435,0.90435,0.90435,-110.0,50.0
2002-5-9,9,5,2002,1,0.90935,0.90935,0.90935,0.90935,50.0,47.0
2002-5-10,10,5,2002,2,0.91405,0.91405,0.91405,0.91405,47.0,-25.0
2002-5-13,13,5,2002,1,0.91155,0.91155,0.91155,0.91155,-25.0,-88.0
2002-5-14,14,5,2002,1,0.90275,0.90275,0.90275,0.90275,-88.0,98.0
2002-5-15,15,5,2002,1,0.91255,0.91255,0.91255,0.91255,98.0,-6.0
2002-5-16,16,5,2002,1,0.91195,0.91195,0.91195,0.91195,-6.0,89.5
2002-5-17,17,5,2002,1,0.9209,0.9209,0.9209,0.9209,89.5,-8.0


In [19]:
sf_eur_usd_features.add_column(predict_pips_list.apply(lambda x: 'sell' if x<0 else 'buy'),'predict action')

date,day,month,year,volume,open,high,low,close,pips,predict pips,predict action
2002-5-6,6,5,2002,1,0.91825,0.91825,0.91825,0.91825,0.0,-29.0,sell
2002-5-7,7,5,2002,1,0.91535,0.91535,0.91535,0.91535,-29.0,-110.0,sell
2002-5-8,8,5,2002,1,0.90435,0.90435,0.90435,0.90435,-110.0,50.0,buy
2002-5-9,9,5,2002,1,0.90935,0.90935,0.90935,0.90935,50.0,47.0,buy
2002-5-10,10,5,2002,2,0.91405,0.91405,0.91405,0.91405,47.0,-25.0,sell
2002-5-13,13,5,2002,1,0.91155,0.91155,0.91155,0.91155,-25.0,-88.0,sell
2002-5-14,14,5,2002,1,0.90275,0.90275,0.90275,0.90275,-88.0,98.0,buy
2002-5-15,15,5,2002,1,0.91255,0.91255,0.91255,0.91255,98.0,-6.0,sell
2002-5-16,16,5,2002,1,0.91195,0.91195,0.91195,0.91195,-6.0,89.5,buy
2002-5-17,17,5,2002,1,0.9209,0.9209,0.9209,0.9209,89.5,-8.0,sell


In [20]:
def calculate_momentum(time_unit):
    close_minus_n = close_minus_n_time_unit(time_unit)
    return close_list - close_minus_n
def calculate_roc(time_unit):
    close_minus_n = close_minus_n_time_unit(time_unit)
    return (close_list - close_minus_n)/close_minus_n

In [21]:
sf_eur_usd_features.add_column(calculate_momentum(3),'momentum_3')
sf_eur_usd_features.add_column(calculate_momentum(4),'momentum_4')
sf_eur_usd_features.add_column(calculate_momentum(5),'momentum_5')
sf_eur_usd_features.add_column(calculate_momentum(8),'momentum_8')
sf_eur_usd_features.add_column(calculate_momentum(9),'momentum_9')
sf_eur_usd_features.add_column(calculate_momentum(10),'momentum_10')
sf_eur_usd_features.add_column(calculate_roc(3),'roc_3')
sf_eur_usd_features.add_column(calculate_roc(4),'roc_4')
sf_eur_usd_features.add_column(calculate_roc(5),'roc_5')
sf_eur_usd_features.add_column(calculate_roc(8),'roc_8')
sf_eur_usd_features.add_column(calculate_roc(9),'roc_9')
sf_eur_usd_features.add_column(calculate_roc(10),'roc_10')
sf_eur_usd_features.add_column(calculate_roc(12),'roc_12')
sf_eur_usd_features.add_column(calculate_roc(13),'roc_13')
sf_eur_usd_features.add_column(calculate_roc(14),'roc_14')
sf_eur_usd_features.add_column(calculate_roc(15),'roc_15')

date,day,month,year,volume,open,high,low,close,pips,predict pips,predict action
2002-5-6,6,5,2002,1,0.91825,0.91825,0.91825,0.91825,0.0,-29.0,sell
2002-5-7,7,5,2002,1,0.91535,0.91535,0.91535,0.91535,-29.0,-110.0,sell
2002-5-8,8,5,2002,1,0.90435,0.90435,0.90435,0.90435,-110.0,50.0,buy
2002-5-9,9,5,2002,1,0.90935,0.90935,0.90935,0.90935,50.0,47.0,buy
2002-5-10,10,5,2002,2,0.91405,0.91405,0.91405,0.91405,47.0,-25.0,sell
2002-5-13,13,5,2002,1,0.91155,0.91155,0.91155,0.91155,-25.0,-88.0,sell
2002-5-14,14,5,2002,1,0.90275,0.90275,0.90275,0.90275,-88.0,98.0,buy
2002-5-15,15,5,2002,1,0.91255,0.91255,0.91255,0.91255,98.0,-6.0,sell
2002-5-16,16,5,2002,1,0.91195,0.91195,0.91195,0.91195,-6.0,89.5,buy
2002-5-17,17,5,2002,1,0.9209,0.9209,0.9209,0.9209,89.5,-8.0,sell


In [22]:
%time
#seems graphlab create bug. takes so long time. deaf lock??
"""
def calculate_fast_k(time_unit):
    #Fast K 100 * [( C - L (n) ) / ( H (n) – L (n) )] . Use the data in same day as initial value
    #L(n) means the lowest Low during the n-day period
    #H(n) means the highest high during the n-day period
    #http://investexcel.net/how-to-calculate-the-stochastic-oscillator/
    high_list = sf_eur_usd_features ['high']
    low_list = sf_eur_usd_features ['low']
    high_high_list =gl.SArray ()
    low_low_list=gl.SArray ()
    for i in range(close_list.size()):
        start = max(0,i-time_unit)
        high_high = gl.SArray ([high_list[start:i+1].max()])
        high_high_list.append(high_high)
        low_low = gl.SArray ([low_list[start:i+1].min()])
        low_low_list.append(low_low)
    print(high_high_list[0:10])
    print(low_low_list[0:10])
calculate_fast_k(3)
close_list.size()
high_python_list = list(sf_eur_usd_features ['high'])
high_python_list[0:1]
"""

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 13.1 µs
[]
[]


[0.91825]

In [24]:
%time

def calculate_fast_k(time_unit):
    #Fast K 100 * [( C - L (n) ) / ( H (n) – L (n) )] . Use the data in same day as initial value
    #L(n) means the lowest Low during the n-day period
    #H(n) means the highest high during the n-day period
    #http://investexcel.net/how-to-calculate-the-stochastic-oscillator/
    high_python_list = list(sf_eur_usd_features ['high'])
    low_python_list = list(sf_eur_usd_features ['low'])
    high_high_list =[]
    low_low_list=[]
    for i in range(close_list.size()):
        start = max(0,i-time_unit)
        high_high = max(high_python_list[start:i+1])
        high_high_list.append(high_high)
        low_low = min(low_python_list[start:i+1])
        low_low_list.append(low_low)
    print(high_high_list[0:10])
    print(low_low_list[0:10])
calculate_fast_k(3)
close_list.size()
high_python_list = list(sf_eur_usd_features ['high'])
high_python_list[0:1]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs
[0.91825, 0.91825, 0.91825, 0.91825, 0.91535, 0.9140499999999999, 0.9140499999999999, 0.9140499999999999, 0.91255, 0.9209]
[0.91825, 0.91535, 0.90435, 0.90435, 0.90435, 0.90435, 0.9027499999999999, 0.9027499999999999, 0.9027499999999999, 0.9027499999999999]


[0.91825]